In [1]:
import pandas as pd

df = pd.read_csv('/content/test_v1.csv')
display(df.head())

,id,instruction,test_list
0,1,স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস...,"['assert remove_Occ(\""hello\"",\""l\"") == \""heo\""']"
1,2,একটি প্রদত্ত ম্যাট্রিক্সকে তার সারিগুলির যোগফল...,"['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1..."
2,3,একটি ফাংশন লিখুন যা একটি অভিধানে সবচেয়ে সাধার...,"['assert count_common([\'red\',\'green\',\'bla..."
3,4,একটি ত্রিভুজাকার প্রিজমের আয়তন খুঁজে বের করার...,"['assert find_Volume(10,8,6) == 240']"
4,5,একটি স্ট্রিংকে ছোট অক্ষরে বিভক্ত করার জন্য একট...,"['assert split_lowerstring(\""AbCd\"")==[\'bC\',..."


In [2]:
df['bengali'] = df['instruction'].str.split('\n', expand=True)[0]
display(df.head())

,id,instruction,test_list,bengali
0,1,স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস...,"['assert remove_Occ(\""hello\"",\""l\"") == \""heo\""']",স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস...
1,2,একটি প্রদত্ত ম্যাট্রিক্সকে তার সারিগুলির যোগফল...,"['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1...",একটি প্রদত্ত ম্যাট্রিক্সকে তার সারিগুলির যোগফল...
2,3,একটি ফাংশন লিখুন যা একটি অভিধানে সবচেয়ে সাধার...,"['assert count_common([\'red\',\'green\',\'bla...",একটি ফাংশন লিখুন যা একটি অভিধানে সবচেয়ে সাধার...
3,4,একটি ত্রিভুজাকার প্রিজমের আয়তন খুঁজে বের করার...,"['assert find_Volume(10,8,6) == 240']",একটি ত্রিভুজাকার প্রিজমের আয়তন খুঁজে বের করার...
4,5,একটি স্ট্রিংকে ছোট অক্ষরে বিভক্ত করার জন্য একট...,"['assert split_lowerstring(\""AbCd\"")==[\'bC\',...",একটি স্ট্রিংকে ছোট অক্ষরে বিভক্ত করার জন্য একট...


In [3]:
import torch
from transformers import pipeline

pipeline = pipeline(task="translation", model="facebook/nllb-200-distilled-600M", src_lang="ben_Beng", tgt_lang="eng_Latn", dtype=torch.float16, device=0)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [4]:
# Processing in chunks
chunk_size = 16
translated_texts = []

for i in range(0, len(df), chunk_size):
    print(i)
    chunk = df['bengali'].iloc[i:i+chunk_size].tolist()
    results = pipeline(chunk)
    translated_texts.extend([result['translation_text'] for result in results])


0
16
32
48
64
80
96
112
128
144


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496


In [5]:
df['remaining_instruction'] = df['instruction'].str.split('\n', expand=True, n=1)[1].fillna('')
df['english_instruction'] = [f"{translated_texts[i]}\n{df['remaining_instruction'][i]}" for i in range(len(translated_texts))]
df = df.drop(columns=['bengali', 'remaining_instruction'])
display(df.head())

,id,instruction,test_list,english_instruction
0,1,স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস...,"['assert remove_Occ(\""hello\"",\""l\"") == \""heo\""']",Type a Python function to remove the first and...
1,2,একটি প্রদত্ত ম্যাট্রিক্সকে তার সারিগুলির যোগফল...,"['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1...",Write a function to arrange a given matrix in ...
2,3,একটি ফাংশন লিখুন যা একটি অভিধানে সবচেয়ে সাধার...,"['assert count_common([\'red\',\'green\',\'bla...",Write a function that counts the most common w...
3,4,একটি ত্রিভুজাকার প্রিজমের আয়তন খুঁজে বের করার...,"['assert find_Volume(10,8,6) == 240']",Write a Python function to find the volume of ...
4,5,একটি স্ট্রিংকে ছোট অক্ষরে বিভক্ত করার জন্য একট...,"['assert split_lowerstring(\""AbCd\"")==[\'bC\',...",Write a function to divide a string into small...


In [6]:
df.to_csv('test.csv', index=False)